## Playing with Austrian athletes and formatting options

Contents
* combine Athletes_01_AUT.ttl and noc_regions.ttl
* how many Austrians took part in OGs
* query an athlete, order by age and see how many times he/she took part in OGs
* format the above output in different ways
* average Age of all Austrian athletes
* oldest and youngest athlete

### Uncomment if not installed

In [98]:
import sys
!{sys.executable} -m pip install rdflib pandas sparqldataframe simplejson

In [99]:
from rdflib import Graph
from pandas import DataFrame
from SPARQLWrapper import SPARQLWrapper, JSON
import sparqldataframe

### Import ttl files

In [100]:
g = Graph()
g.parse("ttl/Athletes_01_AUT.ttl", format="turtle")
print(len(g))

15317


In [101]:
g.parse("ttl/noc_regions.ttl", format="turtle")
print(len(g))

15547


### How many Austrians took part in Olympic Games

In [102]:
result = g.query("""
PREFIX foaf:  <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT (COUNT(DISTINCT ?name) AS ?count)
WHERE {
    ?athlete a foaf:Person;
             rdfs:label ?name;
}
""")

for row in result:
    print("%s Austrian took part" % row)

2334 Austrian took part


### Query Name, order by age and see how many times an Athlete took part in OGs: 

eg: Hubert Raudaschl, Christoph Sumann, Hermann Maier, ...

In [103]:
result = g.query("""
PREFIX foaf:  <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX ex: <http://example.org/ontology/olympics/>
PREFIX noc: <http://example.org/resource/olympics/NOC/>

SELECT ?athleteLabel ?age ?country
WHERE {
    ?athlete a foaf:Person;
             rdfs:label "Hubert Raudaschl";
             foaf:age ?age;
             rdfs:label ?athleteLabel;
             ex:noc ?noc .
    ?noc dbo:country ?country .
}
ORDER BY ?age
""")

print("Total ",len(result))

for row in result:
    print(row)

Total  9
(rdflib.term.Literal('Hubert Raudaschl'), rdflib.term.Literal('22', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#int')), rdflib.term.Literal('Austria'))
(rdflib.term.Literal('Hubert Raudaschl'), rdflib.term.Literal('26', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#int')), rdflib.term.Literal('Austria'))
(rdflib.term.Literal('Hubert Raudaschl'), rdflib.term.Literal('30', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#int')), rdflib.term.Literal('Austria'))
(rdflib.term.Literal('Hubert Raudaschl'), rdflib.term.Literal('33', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#int')), rdflib.term.Literal('Austria'))
(rdflib.term.Literal('Hubert Raudaschl'), rdflib.term.Literal('37', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#int')), rdflib.term.Literal('Austria'))
(rdflib.term.Literal('Hubert Raudaschl'), rdflib.term.Literal('41', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#int')), rdf

#### Format %

In [104]:
print("Total: ",len(result))

for row in result:
    print("Athlete: %s, Age: %s, COO: %s" % row)

Total:  9
Athlete: Hubert Raudaschl, Age: 22, COO: Austria
Athlete: Hubert Raudaschl, Age: 26, COO: Austria
Athlete: Hubert Raudaschl, Age: 30, COO: Austria
Athlete: Hubert Raudaschl, Age: 33, COO: Austria
Athlete: Hubert Raudaschl, Age: 37, COO: Austria
Athlete: Hubert Raudaschl, Age: 41, COO: Austria
Athlete: Hubert Raudaschl, Age: 46, COO: Austria
Athlete: Hubert Raudaschl, Age: 49, COO: Austria
Athlete: Hubert Raudaschl, Age: 53, COO: Austria


#### with index

In [105]:
for index,row in enumerate(result):
    print(index+1, "Athlete: %s, Age: %s, COO: %s" % row)

1 Athlete: Hubert Raudaschl, Age: 22, COO: Austria
2 Athlete: Hubert Raudaschl, Age: 26, COO: Austria
3 Athlete: Hubert Raudaschl, Age: 30, COO: Austria
4 Athlete: Hubert Raudaschl, Age: 33, COO: Austria
5 Athlete: Hubert Raudaschl, Age: 37, COO: Austria
6 Athlete: Hubert Raudaschl, Age: 41, COO: Austria
7 Athlete: Hubert Raudaschl, Age: 46, COO: Austria
8 Athlete: Hubert Raudaschl, Age: 49, COO: Austria
9 Athlete: Hubert Raudaschl, Age: 53, COO: Austria


#### with pandas DataFrame

In [107]:
DataFrame(result, columns=result.vars)

,athleteLabel,age,country
0,Hubert Raudaschl,22,Austria
1,Hubert Raudaschl,26,Austria
2,Hubert Raudaschl,30,Austria
3,Hubert Raudaschl,33,Austria
4,Hubert Raudaschl,37,Austria
5,Hubert Raudaschl,41,Austria
6,Hubert Raudaschl,46,Austria
7,Hubert Raudaschl,49,Austria
8,Hubert Raudaschl,53,Austria


### Average Age of all Austrian athletes

In [108]:
result = g.query("""
PREFIX foaf:  <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT (ROUND(AVG(?age)) AS ?rounded_average_age)
WHERE {
    ?athlete a foaf:Person;
             foaf:age ?age; 
}
""")   
for row in result:
    print("Average Age: %s" % row)

Average Age: 27


### Oldest and youngest athlete

In [109]:
result = g.query("""
PREFIX foaf:  <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT (MAX(?age) as ?maxAge) (MIN(?age) as ?minAge)
WHERE {
    ?athlete a foaf:Person;
             rdfs:label ?name;
             foaf:age ?age;
}
""")   
for row in result:
    print ("Oldest: %s \nYoungest: %s" % row)

Oldest: 76 
Youngest: 12


### SPARQLWrapper

In [115]:
# endpoint = "http://dbpedia.org/sparql"
q = """
    SELECT ?label ?wikiPageID
    WHERE {
    <http://dbpedia.org/resource/Olympics> rdfs:label ?label ;
         <http://dbpedia.org/ontology/wikiPageID> ?wikiPageID
    }
"""

#df = sparql_dataframe.get(endpoint, q)
df = sparqldataframe.dbpedia_query(q)
df.head()

,label,wikiPageID
0,Olympics,19391209


In [116]:
df.describe()

,label,wikiPageID
count,1,1
unique,1,1
top,Olympics,19391209
freq,1,1


In [117]:
df['label'].value_counts()[:10]

Olympics    1
Name: label, dtype: int64

In [74]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

sparql.setQuery("""
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbp:<http://dbpedia.org/property/>
    PREFIX dbo:<http://dbpedia.org/ontology/>

    SELECT ?countryLabel ?capital ?pop
    WHERE {
        ?country a dbo:Country.
        ?country rdfs:label ?countryLabel .
         ?country dbo:capital ?capital.
         ?country dbp:populationCensus | dbo:populationTotal ?pop .
         filter(langMatches(lang(?countryLabel),"EN"))
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["countryLabel"]["value"], result["pop"]["value"] )


Transnistria autonomous territorial unit with special legal status 505153
Transnistria autonomous territorial unit with special legal status 555347
Syria 17064854
Seneca Nation of Indians 8000
Seneca Nation of Indians 8000
Åland Islands 29013
Ascension Island 880
Egypt 85783
Egypt 72798000
United States 324720797
New Zealand 4242048
United States 309349689
Twente 626586
Zeila & Lughaya State 270858
Republic of Saugeais 4337
Afghanistan 32564342
Albania 2886026
Algeria 40400000
Andorra 85470
Anguilla 13600
Antigua and Barbuda 91295
Aruba 103400
Albania 2821977
Algeria 37900000
Angola 25789024
Anguilla 13452
Antigua and Barbuda 81799
Azerbaijan 9754830
Bangladesh 168957745
Belarus 9498700
Belize 368310
Benin 10879829
Bolivia 11410651
Brazil 206440850
Brunei 415717
Burkina Faso 17322796
Cameroon 22534532
Central African Republic 4709000
Chad 13670084
Chile 18006407
China 1376049000
Christmas Island 2072
Cocos (Keeling) Islands 596
Colombia 48786100
Comoros 798000
Croatia 4190700
Cyprus 11